In [1]:
import os 
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

In [27]:
import torch
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# Own library imports
from vecopsciml.utils import TensOps
from vecopsciml.operators.zero_order import Mx, My
from vecopsciml.kernels.derivative import DerivativeKernels

# Function from this project
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop
from utils.fourier_base import compute_fourier_base

# Import model
from architectures.pgnniv_fourier import PGNNIVFourier

In [4]:
# Dataset
dataset = 'non_linear'
N_data = 100
noise = 1

data_name = dataset + '_' + str(N_data) + '_' + str(noise)

In [5]:
# Model
model = 'fourier'
n_modes = 10

model_name = model + '_model_' + str(n_modes)

In [6]:
ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), "../../"))
DATA_PATH = os.path.join(ROOT_PATH, r'data/', data_name, data_name) + '.pkl'
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/', data_name)
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/', data_name, model_name)

In [7]:
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder successfully created at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear_100_1
Folder successfully created at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear_100_1/fourier_model_10


In [8]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear_100_1/non_linear_100_1.pkl


In [9]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [10]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


In [11]:
# Train data splitting in train/test
X = torch.tensor(dataset['X_train'], dtype=torch.float32).unsqueeze(1)
y = torch.tensor(dataset['y_train'], dtype=torch.float32).unsqueeze(1)
K = torch.tensor(dataset['k_train'], dtype=torch.float32).unsqueeze(1)
f = torch.tensor(dataset['f_train'], dtype=torch.float32).unsqueeze(1)

X_train, X_test, y_train, y_test, K_train, K_test, f_train, f_test = train_test_split(X, y, K, f, test_size=0.3, random_state=42)

# Data processing and adequacy with our TensOps library
X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)

y_train = TensOps(y_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_test = TensOps(K_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

f_train = TensOps(f_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_test = TensOps(f_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

# Loading and processing validation data
X_val = torch.tensor(dataset['X_val'], dtype=torch.float32).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

In [28]:
num_modes = 20 

X_mesh = torch.tensor(dataset['X_mesh'])
Y_mesh = torch.tensor(dataset['Y_mesh'])

base = compute_fourier_base(num_modes, X_mesh, Y_mesh).to(DEVICE)

In [29]:
# Predictive network architecture
input_shape = X_train[0].shape
predictive_layers = [20, 10, num_modes]
predictive_output = y_train.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train)).values[0].shape

# Other parameters 
n_filters_explanatory = 5

In [30]:
# Load model and the optimizer
model = PGNNIVFourier(input_shape, predictive_layers, base, predictive_output, explanatory_input, explanatory_layers, explanatory_output, n_filters_explanatory, DEVICE).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 1000

batch_size = 64
n_checkpoints = 5

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
        D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
        model_results_path=MODEL_RESULTS_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 1.591e+09, Test loss: 1.574e+09, MSE(e): 1.562e+02, MSE(pi1): 2.647e+03, MSE(pi2): 3.999e+01, MSE(pi3): 1.762e+01
Epoch 10, Train loss: 4.066e+08, Test loss: 4.468e+08, MSE(e): 4.044e+01, MSE(pi1): 1.795e+02, MSE(pi2): 3.287e+01, MSE(pi3): 3.747e+00
Epoch 20, Train loss: 1.313e+08, Test loss: 1.484e+08, MSE(e): 1.310e+01, MSE(pi1): 2.496e+01, MSE(pi2): 1.122e+01, MSE(pi3): 5.247e-01
Epoch 30, Train loss: 9.892e+07, Test loss: 1.065e+08, MSE(e): 9.886e+00, MSE(pi1): 3.163e+00, MSE(pi2): 6.118e+00, MSE(pi3): 1.907e-01
Epoch 40, Train loss: 6.683e+07, Test loss: 7.455e+07, MSE(e): 6.676e+00, MSE(pi1): 5.079e+00, MSE(pi2): 5.561e+00, MSE(pi3): 1.557e-01
Epoch 50, Train loss: 5.489e+07, Test loss: 7.039e+07, MSE(e): 5.481e+00, MSE(pi1): 5.252e+00, MSE(pi2): 4.035e+00, MSE(pi3): 1.619e-01
Epoch 60, Train loss: 4.865e+07, Test loss: 6.036e+07, MSE(e): 4.859e+00, MSE(pi1): 3.390e+00, MSE(pi2): 3.889e+00, MSE(pi3): 1.627e-01
Epoch 70, Train l

In [26]:
# Parametros de entrenamiento
start_epoch = 800
n_epochs = 2000

batch_size = 64 
n_checkpoints = 10

second_lr = 3e-4

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
        D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
        model_results_path=MODEL_RESULTS_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 800.
Epoch 800, Train loss: 3.937e+06, Test loss: 5.202e+06, MSE(e): 3.890e-01, MSE(pi1): 2.490e+00, MSE(pi2): 2.889e-01, MSE(pi3): 2.119e-01
Epoch 900, Train loss: 3.868e+06, Test loss: 5.113e+06, MSE(e): 3.822e-01, MSE(pi1): 2.489e+00, MSE(pi2): 2.843e-01, MSE(pi3): 2.119e-01
Epoch 1000, Train loss: 3.796e+06, Test loss: 5.020e+06, MSE(e): 3.749e-01, MSE(pi1): 2.489e+00, MSE(pi2): 2.795e-01, MSE(pi3): 2.119e-01
Epoch 1100, Train loss: 3.720e+06, Test loss: 4.921e+06, MSE(e): 3.673e-01, MSE(pi1): 2.488e+00, MSE(pi2): 2.743e-01, MSE(pi3): 2.119e-01
Epoch 1200, Train loss: 3.641e+06, Test loss: 4.819e+06, MSE(e): 3.594e-01, MSE(pi1): 2.487e+00, MSE(pi2): 2.690e-01, MSE(pi3): 2.119e-01
Epoch 1300, Train loss: 3.559e+06, Test loss: 4.713e+06, MSE(e): 3.513e-01, MSE(pi1): 2.487e+00, MSE(pi2): 2.635e-01, MSE(pi3): 2.119e-01
Epoch 1400, Train loss: 3.475e+06, Test loss: 4.603e+06, MSE(e): 3.429e-01, MSE(pi1): 2.487e+00, MSE(pi2): 2.577e-01, MSE(pi3)